# RAG (Retrieval-Augmented Generation)

## 1. 서론: 왜 RAG인가?

### 1.1 LLM의 한계

최근 인공지능 분야에서 **대규모 언어 모델(LLM)** 은 자연어 처리, 텍스트 생성, 번역, 챗봇 등 다양한 분야에서 뛰어난 성능을 보여주고 있습니다.  
그러나 LLM에도 몇 가지 **한계**가 존재합니다:

- **환각(Hallucination)**  
  모델이 실제 사실과 다르거나 잘못된 정보를 생성하는 현상  
  - 예: 2025년 노벨 문학상 수상자는 …라는 질문에 실제 수상자와 다른 답변을 생성

- **지식 업데이트 지연**  
  LLM은 `사전 학습된 데이터에 기반`하므로, 최신 정보가 반영되지 않을 수 있음  
  - 예: 최근 출시된 제품이나 논문 정보는 모델이 알지 못함

- **희소 정보(Sparse Knowledge)에 대한 취약성**  
  학습 데이터에서 드물게 등장하는 내용에 대해 정확한 답변을 생성하지 못함  
  - 예: 거의 알려지지 않은 전자기기 기능, 일부 특수 분야의 세부 매뉴얼  
  - 예: 대중적으로 잘 알려지지 않은 개인에 대한 질문

- **투명성 부족**  
  모델이 왜 특정 답변을 생성했는지 이해하거나 추적하기 어려움  
  - 예: 단순히 출력 결과만 확인 가능, 내부 근거는 불명확


### 1.2 검색(Retrieval)과 생성(Generation) 결합 필요성

LLM의 한계(최신 정보 부족, 희소 정보 취약, 환각 등)를 보완하기 위해 **검색(Retrieval)** 과 **생성(Generation)** 을 결합하는 방식이 등장했습니다.  

이때 검색(Retrieval)은 크게 아래 두 가지로 나누어 이해할 수 있습니다.
  1. **웹 검색(Web Retrieval)**
  2. **문서 검색(Document Retrieval)**

#### 1) 웹 검색(Web Retrieval)
- 인터넷에서 **최신 정보**를 실시간으로 조회  
- 최근 뉴스, 연구논문, 신제품, 변경된 규정 등 모델이 학습하지 못한 업데이트된 사실을 바로 반영 가능  
  - 예: "지금 대통령이 누구인가요?" 질문에 정확한 답변 가능
- LLM의 지식 업데이트 지연 문제 보완

#### 2) 문서 검색(Document Retrieval)
- 내부 DB, PDF, 매뉴얼, 논문, 회사 문서 등에서 **희소 정보(sparse knowledge)** 나 **전문 지식**을 찾아 제공  
- 학습 데이터에 거의 등장하지 않는 정보도 정확하게 제공  
  - 예: 특정 기업 내부 규정, 특수 장비 사용 설명서, 비공개 연구 자료 등  
- LLM의 특수·희소 정보 취약 문제 보완

#### 3) 생성(Generation)
- **검색된 문서·웹 정보**를 기반으로 자연스럽고 일관된 답변을 생성  
- 단순히 검색 결과를 나열하는 것이 아니라 **요약·통합·맥락 반영**까지 수행


#### 결론
> **웹 검색 → 최신 정보 보완**  
> **문서 검색 → 희소·전문 지식 보완**  
> **생성 → 자연스럽게 정리된 답변 제공**

이렇게 검색과 생성을 결합한 구조가 바로  **RAG: Retrieval-Augmented Generation**이며  
LLM의 현실적 한계를 극복하는 핵심 기술로 자리 잡고 있습니다.


#### 예시 및 정리

예를 들어, **자동차 제조 공장**에서 설비 위치를 파악하고 싶다고 가정해봅시다.

- 질문: "C공장 3라인의 용접 로봇 R-12 위치가 어디인가요?"  
- 단순 LLM(예: ChatGPT)만으로는 공장 내부 설비 위치와 같은 세부 정보는 알 수 없습니다.  
- **RAG 접근**:  
  1. **검색(Retrieval)** 단계에서 최신 공장 설계도, 설비 배치 문서, 유지보수 기록 등을 찾아 관련 정보를 확보합니다.  
  2. **생성(Generation)** 단계에서 검색된 설비 배치 정보를 바탕으로, "C공장 3라인의 용접 로봇 R-12는 조립 구역 중앙, 검사 라인 옆에 위치해 있습니다."처럼 자연스럽게 정리된 답변을 제공합니다.  

이 방식은 단순히 문서를 보여주는 것이 아니라, 사용자가 실제로 필요로 하는 정보를 맥락에 맞게 **정리·통합**해서 전달할 수 있습니다는 장점이 있습니다.

### 1.3 RAG 구성 요소

RAG(Retrieval-Augmented Generation)는 기본적으로 세 가지 핵심 요소로 구성됩니다:

- **Retriever(검색기)**  
  질문과 관련된 문서나 정보를 실시간으로 검색
- **Generator(생성기)**  
  검색된 정보를 바탕으로 자연스럽게 답변 생성
- **Augmentation 방법**  
  검색 결과를 프롬프트에 결합하여 생성 모델이 활용할 수 있도록 통합

💡 직관적 비유  
- 기존 LLM: 머릿속 지식만으로 답변하는 사람  
- RAG: 도서관에서 자료를 찾아 확인하고 답변하는 사람

#### RAG 구조 도식

<img src="image/rag_example.png" width="500"> 
  
이미지 출처 : https://valueminer.eu/retrieval-augmented-generation-rag/

### 1.4 RAG의 핵심 요약 및 산업적 필요성

#### RAG가 필요한 기술적 이유 (요약)
- LLM은 환각(Hallucination), 최신 정보 반영 불가, 희소 정보 취약 등의 구조적 한계를 지님  
- 웹 검색(Web Retrieval)을 통해 최신 정보를 보완  
- 문서 검색(Document Retrieval)을 통해 희귀·전문·내부 문서를 반영  
- 생성(Generation)은 검색 결과를 바탕으로 맥락 있게 정리·통합해 최종 답변 생성

요약하면, **검색으로 사실 확보 + 생성으로 자연스러운 답변 생성**이라는 조합이  
기존 LLM이 가진 본질적인 약점을 가장 효과적으로 해결한다.

#### 왜 산업 현장에서 RAG가 더 중요해졌는가?

현업에서는 LLM의 정확성·근거·최신성이 필수인데, 단순 모델만으로는 이를 충족하기 어렵기 때문입니다.

특히 다음과 같은 이유로 RAG는 필수 기술로 자리 잡고 있음:

- **기업 내부 문서 활용 필요**  
  기술 매뉴얼, 고객 DB, 설비 정보, 규정집 등  
  모델이 학습하지 못한 내부 지식을 실시간으로 반영 가능

- **최신 정보 의존 서비스**  
  고객센터, 물류, 금융, 공공 행정 등  
  매일 바뀌는 정책·재고·가격 등을 최신 상태로 유지해야 하는 분야에서 필수

- **정확성이 생명인 산업**  
  제조, 의료, 법률, 정비, 품질관리처럼  
  잘못된 답변이 사고/금전적 손실로 이어지는 분야에서 신뢰도 향상

#### 보안 때문에 오픈소스 모델 + RAG 조합이 더욱 확산되는 이유

많은 기업들은 다음 이유로 ChatGPT 같은 상용 모델을 그대로 쓰기 어렵다:

- 민감한 정보를 외부 서버로 보내기 어려움 (보안/규정)  
- 클라우드 기반 모델의 개인정보 규제가 강화  
- 데이터가 외부로 유출될 가능성을 원천 차단하고 싶음  

그래서 기업들은 성능은 다소 낮지만, **로컬 또는 사내 서버에서 직접 실행 가능한 오픈소스 모델(Llama, Gemma 등)** 을 선호하는데,  
이때 부족한 성능을 보완하기 위한 핵심 전략이 바로 **RAG**다.

> 오픈소스 모델 + 사내 문서 RAG => **보안 유지 + 높은 정확도 + 최신 정보 반영** 을 동시에 만족시키는 사실상 업계 표준 조합


#### 결론

RAG는 단순한 LLM 보조 기술이 아니라 **기업 환경에서 신뢰도·보안·업무 정확성을 보장하기 위한 필수 기술 스택**입니다.  
고성능 상용 모델은 물론, 사내에서 운영하는 오픈소스 모델에서도  
RAG가 결합될 때 비로소 실무에서 사용할 수 있는 수준의 품질을 확보할 수 있습니다.


## 2. RAG 기본 이해

### 2.1 정의와 개념

**RAG (Retrieval-Augmented Generation)** 은 LLM의 답변 품질을 높이기 위해  
모델 외부에서 필요한 정보를 찾아(Look-up) 이를 바탕으로 생성(Generate)하는 기술입니다.

- **핵심 개념**  
  - LLM이 가진 사전 지식만으로 답변하지 않고  
  - **검색을 통해 확보한 최신·정확한 정보를 함께 참조하여 답변을 생성하는 구조**

- **해결하는 문제**  
  - 환각(Hallucination) 완화  
  - 최신 정보 반영  
  - 답변 출처 및 근거 제시 → 투명성 확보

- **적용 분야**  
  - 고객지원 챗봇, 기술문서 Q&A  
  - 사내 문서 기반 정보 검색 시스템  
  - 법률/의료/제조 등 근거 기반 응답이 필요한 영역  
  - 실시간 정보 기반 보고서·데이터 분석

> 요약: RAG는 검색으로 사실 확보 + 생성으로 답변 구성이라는 두 단계를 결합해  
> LLM의 한계를 보완하고 **정확성·신뢰성·업무 적용성**을 강화하는 기술입니다.


### 2.2 핵심 구성 요소

RAG는 크게 세 가지 요소로 구성됩니다.

<img src="image/RAG.png" width="450">

1. **Retriever (검색기)**  
   - 사용자의 질문과 의미적으로 가까운 문서·문장·데이터를 검색  
   - 이때 **Embedding + Vector DB(FAISS, Chroma 등)** 를 활용하여 의미 기반 검색을 수행  
   - 검색 품질이 최종 답변의 정확도에 직접적인 영향
2. **Augmentation Methods (증강 방법)**  
   - 검색된 문서를 LLM 입력(prompt)과 결합하는 방식  
   - 단순 삽입, 우선순위 정렬 등 다양한 방식 존재

3. **Generator (생성기)**  
   - 검색된 문서를 바탕으로 자연스럽고 일관된 텍스트 생성  
   - GPT, Llama 등 LLM이 Generator 역할을 수행  
   - 단순 요약이 아니라 **문서 통합·정리·맥락 반영**까지 수행


> Retriever는 ‘어떤 문서를 가져올 것인가’,  
> Generator는 ‘가져온 문서로 어떤 답변을 만들 것인가’를 담당합니다.

### 2.3 벡터 임베딩과 의미 기반 검색의 원리

RAG에서 검색(Retrieval)은 단순한 키워드 검색이 아니라, **텍스트의 의미(Semantics)** 를 기반으로 가장 관련성 높은 문서를 찾아오는 과정입니다.

이를 가능하게 하는 핵심 기술이 바로 **벡터 임베딩(Vector Embedding)** 과 **벡터 데이터베이스(Vector Database)** 입니다.

LLM 기반 검색 시스템이 어떻게 문서를 의미적으로 찾는지 이해하려면 다음 개념들을 알아야 합니다.

#### 1) 전통적 검색(RDBMS)의 한계

<img src="image/RDBMS.png" width="600">  

이미지 출처 : https://www.nilebits.com/blog/2023/03/key-differences-between-oracle-sql-server-mysql-and-postgresql/

전통적 DB는 데이터를 **표 형태(행·열)** 로 저장하고,정확한 값 매칭 기반으로 검색합니다.

예:
- "강아지" != "개" → 단어가 다르면 다른 데이터
- "노트북이 꺼진다" vs "컴퓨터가 종료됩니다" → 단어로 보면 유사성 없음

➡ **의미적 연관성 파악 불가**  
➡ 비정형 데이터(텍스트·이미지) 검색에 부적합


#### 2) 벡터 임베딩과 벡터 데이터베이스(Vector DB)

벡터 임베딩은 텍스트·이미지·오디오 등을 **수백~수천 차원의 숫자 벡터로 변환하는 기술**입니다.  
이 벡터는 단순한 숫자 배열이 아니라 **데이터의 의미적 특징을 표현한 방향 벡터**이며,  
두 벡터의 유사성은 주로 다음과 같은 방식으로 계산됩니다:

- **Cosine Similarity**: 두 벡터의 방향 유사도  
- **Euclidean Distance(L2)**: 벡터 간 거리

예시:

"고양이가 물을 마신다" → [0.12, 0.03, ..., 0.76]  
"강아지가 물을 마신다" → [0.15, 0.01, ..., 0.70]

두 문장은 의미적으로 비슷하기 때문에 **벡터 공간에서도 서로 가깝게 위치**한다. 

##### 벡터 DB(Vector Database)는 무엇을 하는가?

벡터 DB는 이러한 임베딩 벡터를 **효율적으로 저장하고, 의미적 유사성 기반으로 검색(k-NN)** 하기 위한 전용 데이터베이스입니다.

**일반 DB와의 차이**
- 일반 DB: 키-값, 정렬 기반 검색(SQL 등)
- 벡터 DB: 고차원 벡터 공간에서 의미적 유사성 중심 검색

**주요 기능**
- 벡터 추가/삭제  
- 유사 벡터 검색(K-Nearest Neighbor)  
- 벡터와 연결된 메타데이터 저장(ID, 태그, 원문 등)

<img src="image/vectorDB.webp" width=500> 

이미지 출처 : https://www.brilworks.com/blog/what-is-vector-database-types-uses/

##### 임베딩과 벡터 DB의 관계 (핵심 정리)

- **임베딩(Embedding)** → 데이터를 의미 기반 숫자 벡터로 변환  
- **벡터 DB(Vector DB)** → 이 벡터들을 저장하고, 의미적으로 가까운 벡터를 빠르게 검색  

즉,

> **임베딩은 ‘의미를 숫자로 바꾸는 기술’,  
> 벡터 DB는 ‘그 숫자를 저장하고 검색하는 시스템’.**

둘을 합쳐야 의미 기반 검색(Semantic Search)과 RAG의 핵심 기능이 구현됩니다.

#### 3) 벡터 데이터베이스(Vector DB)의 역할

벡터 데이터베이스(Vector DB)는 텍스트·이미지 등을 임베딩한 ‘벡터’를 효율적으로 저장하고,  
의미 기반 검색(k-NN)을 빠르게 수행하는 전용 데이터베이스입니다.

##### 대표적인 벡터 DB 비교

| DB 이름 | 특징 | 장점 | 단점 |
|--------|------|--------|--------|
| **FAISS** | Meta 개발, GPU 지원 | 빠름, 무료, 로컬 쉬움 | 분산 구성 어려움 |
| **Pinecone** | 클라우드 기반 SaaS | 자동 확장, 운영 편리 | 비용 발생 |
| **Weaviate** | 오픈소스·클라우드 | 텍스트·이미지·멀티모달 | 설정 약간 복잡 |
| **Chroma** | 오픈소스 | 간단, 로컬 친화 | 매우 대규모에 한계 |

➡ 대부분의 튜토리얼에서는 **FAISS or Chroma**로 시작  
➡ 기업 서비스 운영에서는 **Pinecone/Weaviate** 활용 증가

##### **벡터 DB가 저장하는 기본 구조**
- 텍스트를 임베딩하면 보통 다음과 같은 **쌍 구조(pair)** 로 저장:

```
{
    "id": "문서 또는 청크 ID",
    "vector": [0.12, -0.88, 0.33, ...],   # 임베딩 벡터
    "text": "원본 텍스트",               # LLM이 참고할 실제 내용
    "metadata": { ... }                  # 출처, 태그, 파일명 등 추가 정보
}
``` 
> **원본 텍스트 + 메타데이터 + 임베딩 벡터**가 함께 저장되며,  
> 검색 시에는 벡터를 이용해 후보를 찾고, 결과 반환 시에는 텍스트와 메타데이터를 함께 제공한다.

#### 4) 벡터 검색 전체 과정

예시 질문: 고양이가 음료를 마신다

벡터 검색은 크게 다음 5단계를 거쳐 이루어집니다.

##### **1. 문서 임베딩 생성**  
검색 대상(문서·문장·데이터셋 등)을 먼저 **숫자 벡터(임베딩)** 로 변환한다.  
이 과정은 사전에 한 번 수행되며, (벡터 + 텍스트 + 메타데이터)가 벡터 DB에 저장됩니다.

- 예:  
  - "고양이가 물을 마신다" → [0.12, 0.03, …]  
  - "강아지가 물을 마신다" → [0.15, 0.01, …]

➡ 사람이 읽는 문장을 컴퓨터가 이해할 수 있는 **의미 기반 좌표값**으로 바꾸는 단계.

##### **2. 쿼리 임베딩 생성**
사용자가 입력한 질문도 동일한 방식으로 벡터화한다.

- 예:  
  - 질문 "고양이가 음료를 마신다" → [0.13, 0.02, …]

➡ 쿼리가 문서와 같은 의미 공간 안에 위치해야 유사성 비교가 가능함.


##### **3. k-NN 검색(k-Nearest Neighbors)**
쿼리 벡터와 **방향적으로 가장 가까운 벡터 K개**를 벡터 DB에서 찾는다.

- K=2라면 → 가장 유사한 문서 2개 반환  
- 유사성 계산 방식:  
  - **Cosine Similarity(방향 유사도)**  
  - Euclidean Distance(거리 기반)

➡ 질문과 의미적으로 가장 비슷한 후보 문서를 가져오는 핵심 단계.


##### **4. 유사 문서 반환**  
k-NN 검색으로 찾은 벡터에 연결된 **원본 문서**를 반환한다.

예시 결과:
- "고양이가 물을 마신다"
- "강아지가 물을 마신다"

➡ 모델은 이제 "문서를 참고할 수 있는 상태"가 됩니다.



##### **5. LLM이 반환 문서를 참고하여 답변 생성**  
Retriever가 넘겨준 문서를 Prompt에 포함시켜 LLM에게 전달한다.  
LLM은 문서를 읽고 그 근거를 바탕으로 **더 정확한 답변을 생성**한다.

예:
> "고양이가 음료를 마시는 상황은 물을 마시는 행동과 유사합니다…"  
(문서 기반 근거 포함)

➡ 단순한 추측이 아니라, 실제 검색된 문서 근거를 바탕으로 답변 생성

### 2.4 요약

- **Retriever(검색기)**  
  질문과 의미가 가장 가까운 문서를 찾아오는 단계  
- **검색 품질 = RAG 전체 성능의 핵심**  
  잘못된 문서를 가져오면 생성 모델도 오류를 전달할 수밖에 없음

#### 5) 왜 벡터 검색이 중요한가?

- 환각(Hallucination) 감소  
- 희소·전문 정보 보완  
- 최신 정보 반영  
- 오픈소스 LLM의 부족한 사전지식 보완  
- 문서 기반 답변 → 신뢰도 향상

➡ 요약: 의미 기반 검색은 RAG 성능을 결정하는 핵심 요소이며
Retriever의 품질이 곧 RAG 전체 품질을 좌우한다.


#### 6) 기본 RAG 코드 예시 (FAISS 기반)

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

# 저장할 문서 목록 (RAG에서 지식 베이스 역할)
docs = [
    "고양이가 물을 마신다",
    "강아지가 물을 마신다",
    "사람이 커피를 마신다",
    "눈에 보이지 않는 것의 엄청난 힘",
    "메리 크리스마스"
]

# OpenAIEmbeddings()
# - 텍스트를 의미 기반 벡터로 변환하는 임베딩 모델.
# - 아무 옵션을 지정하지 않으면 디폴트 모델(text-embedding-3-small 등)이 자동 사용됨.
#   예: OpenAIEmbeddings(model="text-embedding-3-large")처럼 원하는 모델을 지정할 수 있음.
# - 임베딩 벡터는 문장의 의미를 숫자로 표현한 결과이며,
#   유사 문장들은 벡터 공간에서 가깝게 위치하도록 학습되어 있음.
emb = OpenAIEmbeddings()

# FAISS.from_texts()
# - 문장을 임베딩하고, 생성된 벡터를 FAISS 벡터 DB에 저장.
# - 내부적으로 수행되는 과정:
#   1) 각 문장을 임베딩 모델로 벡터화
#   2) (벡터 + 원본 텍스트 + 메타데이터) 형태로 DB에 저장
db = FAISS.from_texts(docs, emb)

# similarity_search(query, k)
# - 사용자의 질문(query)을 동일한 임베딩 모델로 벡터화한 뒤,
#   벡터 DB 안의 벡터들과 비교하여 가장 유사한 k개를 검색.
# - 단어가 비슷한지가 아니라 의미가 얼마나 가까운지를 기준으로 찾는 것이 핵심.
query = "고양이가 음료를 마신다"
results = db.similarity_search(query, k=3)

# 검색된 문서의 원본 텍스트(page_content)를 출력
# 벡터 DB는 벡터만 반환하는 것이 아니라, 벡터와 연결된 원문 텍스트를 함께 반환해야
# RAG에서 LLM이 이를 그대로 참고할 수 있음.
for r in results:
    print(r.page_content)


고양이가 물을 마신다
강아지가 물을 마신다
사람이 커피를 마신다


In [2]:
# 벡터DB 내부 구조 확인
index = db.index

# 총 몇 개의 벡터가 저장되어 있는지
print("저장된 벡터 개수:", index.ntotal)

저장된 벡터 개수: 5


In [3]:
# 벡터의 차원 수(embedding size)
print("벡터 차원:", index.d)

벡터 차원: 1536


In [4]:
# 첫 번째 벡터 가져오기 (0번째 위치)
vector = index.reconstruct(0)
print("첫 번째 벡터 일부:", vector[:10])  # 앞 10개만 출력

첫 번째 벡터 일부: [-0.00387605 -0.02365548  0.00721769 -0.01272271 -0.01931585  0.02351383
 -0.02606352 -0.01355973 -0.01361124 -0.01318629]


In [5]:
# LangChain의 FAISS는 docstore 안에 원문을 저장함
print("문서 ID 리스트:", db.docstore._dict.keys())

문서 ID 리스트: dict_keys(['487c39e9-2dac-42d6-8d74-ae470f4a8076', 'cfd37052-8515-4dfb-8a4d-cbf02073e592', '6d83c7ac-485c-41af-b254-d4d795a4ad5e', 'bc034298-b79e-4b34-9b12-7c51be43f574', '8670f65a-c6b8-409f-a111-cfe7c65cb9fe'])


In [6]:
# 가장 첫 번째 문서 ID 가져오기
first_id = list(db.docstore._dict.keys())[0]
# 해당 ID의 문서 객체 확인
print("첫 문서 내용:", db.docstore._dict[first_id].page_content)

첫 문서 내용: 고양이가 물을 마신다


In [7]:
print("=== 저장된 모든 문서와 메타데이터 ===")
for doc in db.docstore._dict.values():
    print("문서 내용:", doc.page_content)
    print("메타데이터:", doc.metadata)
    print("---")

=== 저장된 모든 문서와 메타데이터 ===
문서 내용: 고양이가 물을 마신다
메타데이터: {}
---
문서 내용: 강아지가 물을 마신다
메타데이터: {}
---
문서 내용: 사람이 커피를 마신다
메타데이터: {}
---
문서 내용: 눈에 보이지 않는 것의 엄청난 힘
메타데이터: {}
---
문서 내용: 메리 크리스마스
메타데이터: {}
---


➡ 의미적 유사성 기반 검색이 정상적으로 작동

#### 벡터 DB 저장 및 불러오기 (Persistence)

매번 코드를 실행할 때마다 벡터 DB를 새로 생성하면 어떻게 될까요?  
수많은 텍스트를 벡터로 변환하는 **임베딩(Embedding)** 과정이 반복되면서 **API 호출 비용이 낭비**되고, 데이터가 많을 경우 **생성 시간도 매우 오래 걸립니다.**  

따라서, 한 번 생성된 벡터 DB는 **로컬 파일로 저장**해두고, 필요할 때 불러와서 재사용하는 것이 효율적입니다.  


LangChain의 `save_local("폴더명")` 함수를 실행하면 해당 폴더 안에 두 개의 핵심 파일이 생성됩니다.

- **`index.faiss`**: 벡터 연산을 위한 실제 인덱스 데이터 (벡터 정보)
- **`index.pkl`**: 매핑된 원본 텍스트(Document)와 메타데이터 정보 (직렬화된 파일)

In [11]:
DB_PATH = "./my_faiss_index_test"
db.save_local(DB_PATH)

print(f"'{DB_PATH}' 경로에 저장 완료되었습니다.")

'./my_faiss_index_test' 경로에 저장 완료되었습니다.


In [10]:
# 로컬에서 불러오기
# 주의: 저장할 때 썼던 임베딩 모델(emb)과 동일한 객체를 넘겨줘야 합니다.
new_db = FAISS.load_local(
    folder_path=DB_PATH, 
    embeddings=emb, 
    allow_dangerous_deserialization=True  # 최신 버전 필수 옵션
)

print("DB 로드 완료!")

# 4. 잘 불러와졌는지 테스트
query = "테스트 질문"
docs = new_db.similarity_search(query)
print(docs[0].page_content)

DB 로드 완료!
사람이 커피를 마신다


## 3. RAG 시스템 구축과 실습

이 장에서는 RAG 구현의 핵심 구성 요소인  
**임베딩 → 벡터 데이터베이스 → 의미 기반 검색 → LLM 생성(Augmented Generation)**  
의 전체 흐름을 실제 구조와 함께 학습합니다.


> 💡 참고: RAG 구축을 위한 프레임워크는 LangChain 외에도 **LlamaIndex**가 있음  
> - LlamaIndex는 문서 인덱싱과 질의 응답 중심에 특화된 프레임워크  
> - 목적과 프로젝트에 따라 적합한 도구 선택 가능

### 3.1 RAG 파이프라인 전체 개요

RAG 시스템은 다음 네 단계로 구성됩니다:

1) **문서 수집(Load)**  
   - PDF, TXT, Word, 웹 페이지, DB, 내부 문서 등

2) **임베딩 생성(Embed)**  
   - 텍스트를 수백~수천 차원의 벡터로 변환  
   - 동일한 의미 공간(Semantic Space)에서 비교를 가능하게 함

3) **벡터 DB 저장(Store)**  
   - FAISS, Chroma, Pinecone 등  
   - 대규모 문서를 빠르게 의미 기반 검색 가능하게 함

4) **검색 후 생성(Retrieve → Generate)**  
   - LLM이 검색된 문서를 바탕으로 답변 생성  
   - RAG의 핵심: 검색된 사실 기반의 생성

### 3.2 예제: 실제 PDF 문서를 활용한 RAG 질의응답 (LCEL 체인 vs 일반 LLM 비교)

아래 예제는 다음 흐름을 보여줍니다.

1. `농업기술.pdf`를 로드하고 토막(T chunk) 단위로 분할  
2. 임베딩 + FAISS 벡터DB에 저장  
3. **LCEL 문법으로 RAG 체인**을 구성해 질의응답 수행  
4. **같은 질문을 RAG 없이 일반 LLM에만 물어본 결과**와 비교  

질문 예시:

> "나무-FRP 복합기둥 공법을 적용했을 때 20년 후 예측 침하량(㎜)과 허용 기준(㎜)은 각각 얼마인가요?"

이 질문은 농촌진흥청 PDF(농업기술) 안에 있는 표를 보지 않으면 답을 알기 어렵기 때문에,  
**RAG가 희소한 도메인 정보를 얼마나 잘 보완해주는지**를 보여주기에 적합합니다.

In [13]:
# 필요한 패키지 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

import os
from dotenv import load_dotenv

# 0. 환경 변수 로드 (OPENAI_API_KEY는 .env에 저장했다고 가정)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요."

In [17]:
# 1. 문서 로드 & 청크 분할
PDF_PATH = "data/농업기술.pdf"  
DB_PATH = "./my_faiss_index"

# 임베딩 모델 준비
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 DB 존재 여부 확인 및 분기 처리
if os.path.exists(DB_PATH):
    # (A) 이미 저장된 DB가 있는 경우 -> 불러오기
    print(f"기존 벡터 DB를 '{DB_PATH}'에서 불러옵니다...")
    
    vectorstore = FAISS.load_local(
        folder_path=DB_PATH, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True # 필수 설정
    )
    print("DB 로드 완료!")

else:
    # (B) 저장된 DB가 없는 경우 -> 새로 생성 및 저장
    print("저장된 DB가 없습니다. 문서를 로드하고 새로 생성합니다...")

    # --- 문서 로드 & 청크 분할 (기존 코드) ---
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n\n", "\n", " ", ""],
    )

    split_docs = text_splitter.split_documents(docs)

    # 메타데이터 ID 부여
    for idx, d in enumerate(split_docs):
        d.metadata["id"] = idx

    print(f"원본 페이지 수: {len(docs)}")
    print(f"분할된 청크 수: {len(split_docs)}")
    
    # --- 벡터 DB 생성 및 저장 ---
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    # 로컬에 저장
    vectorstore.save_local(DB_PATH)
    print(f"벡터 DB가 '{DB_PATH}' 경로에 저장되었습니다.")

# Retriever 생성 (RAG에서 사용할 검색기)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 관련도 높은 청크 4개 정도 가져오기
)

저장된 DB가 없습니다. 문서를 로드하고 새로 생성합니다...
원본 페이지 수: 16
분할된 청크 수: 29
💾 벡터 DB가 './my_faiss_index' 경로에 저장되었습니다.


In [ ]:
# 3. LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",   # 필요 시 다른 모델로 교체 가능
    temperature=0.1,       # 최대한 사실 위주의 응답을 위해 낮게 설정
)

In [ ]:
# 4. 컨텍스트 포맷 함수 (검색된 문서 → 문자열)
def format_docs(docs):
    """검색된 Document 리스트를 하나의 문자열로 합치는 함수."""
    parts = []
    for d in docs:
        # 페이지 정보가 있으면 함께 표시 (디버깅/검증용)
        page = d.metadata.get("page", "N/A")
        parts.append(f"[page {page}]\n{d.page_content}")
    return "\n\n".join(parts)

In [ ]:
# 5. RAG용 프롬프트 & 일반 LLM 프롬프트 정의

# (1) RAG 체인용 프롬프트
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "너는 농촌진흥청에서 발행한 '농업기술' PDF 문서만을 근거로 답변하는 조수야. "
                "반드시 제공된 컨텍스트(context) 안에서만 근거를 찾아 대답해야 하며, "
                "컨텍스트에 정보가 없으면 모른다고 말해야 한다.\n\n"
                "답변에는 가능하면 수치(단위 포함)와 근거가 된 페이지 번호를 함께 언급해 줘."
            ),
        ),
        (
            "user",
            (
                "질문: {question}\n\n"
                "다음은 검색된 참고 컨텍스트야:\n"
                "--------------------\n"
                "{context}\n"
                "--------------------\n"
                "위 컨텍스트만 근거로 질문에 답해줘."
            ),
        ),
    ]
)

# (2) 일반 LLM(비-RAG) 프롬프트
no_rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "너는 일반적인 한국어 어시스턴트야. "
                "인터넷과 일반 상식을 바탕으로 최대한 그럴듯한 답을 해줘."
            ),
        ),
        ("user", "질문: {question}"),
    ]
)

In [ ]:
# 6. LCEL로 RAG 체인 구성
# 입력: {"question": "..."}
# 1) question을 그대로 전달하여 retriever에서 관련 문서 검색
# 2) 검색된 문서들을 format_docs로 합치고 context 필드에 넣음
# 3) rag_prompt로 LLM 호출
# 4) 문자열로 파싱

rag_chain = (
    {
        "question": RunnablePassthrough(),                 # 그대로 통과
        "context": RunnablePassthrough()                   # question을 retriever에 넣기 위한 경로
        | retriever                                        # 관련 문서 검색
        | RunnableLambda(format_docs),                     # 검색 결과를 문자열로 변환
    }
    | rag_prompt                                           # 프롬프트 템플릿 적용
    | llm                                                  # LLM 호출
    | StrOutputParser()                                    # ChatMessage → 문자열
)

# 7. LCEL로 "일반 LLM만 사용하는 체인" 구성 (비교용)
no_rag_chain = (
    no_rag_prompt
    | llm
    | StrOutputParser()
)

# 8. 예시 질문 설정
question = (
    "농촌진흥청 '농업기술' 자료에서 소개된 "
    "나무-FRP 복합기둥 공법의 20년 후 예측 침하량(㎜)과 "
    "허용 기준 침하량(㎜)은 각각 얼마인가요?"
)

# 9. 체인 실행 (RAG vs 비-RAG 비교)
print("=== [1] RAG 체인 결과 ===")
rag_answer = rag_chain.invoke(question)
print(rag_answer)

print("\n\n=== [2] RAG 없이 LLM만 사용한 결과 ===")
no_rag_answer = no_rag_chain.invoke({"question": question})
print(no_rag_answer)

In [15]:
DB_PATH = "./my_faiss_index"
vectorstore.save_local(DB_PATH)

print(f"'{DB_PATH}' 경로에 저장 완료되었습니다.")

'./my_faiss_index' 경로에 저장 완료되었습니다.


※ 기대되는 차이 (실제 출력은 모델/버전마다 달라질 수 있음)

- RAG 체인:
  - PDF 표 내용을 근거로
    - "20년 후 예측 침하량은 약 0.52mm이고, 허용 기준은 25.4mm" 와 같이
      구체적인 수치를 제시하고,
    - "복합기둥 공법 예측 침하량 표 (예: page 10 인근)" 등의 근거를 함께 언급할 가능성이 높음.

- 일반 LLM(비-RAG):
  - 문서 내용을 모르는 상태라서
    - 대략적인 추정치(예: '수밀리미터 수준' 등)를 말하거나,
    - 전혀 다른 수치를 할루시네이션으로 지어낼 수 있음.
  - 특정 페이지나 표를 정확히 인용하기 어려움.

이 예제를 통해
- '문서에만 있는 희소한 정보'를 다룰 때
- 단순 LLM 호출보다 RAG 구조가 훨씬 안정적이고 신뢰도 높은 답변을 줄 수 있습니다는 점을
직접 비교해 볼 수 있습니다.


## 4. Naive RAG의 한계

뒤에 등장할 Advanced RAG를 이해하기 위해서는 먼저 **Naive RAG가 왜 충분하지 않은지**,  
각 단계별로 어떤 문제가 발생하는지 명확히 짚고 넘어갈 필요가 있습니다.

### 4.1 Indexing 단계의 한계

##### **① 정보 추출의 불안정성**
PDF, 이미지 기반 스캔, 표(table) 등 비정형 문서에서 텍스트 추출이 완벽하지 않다.  
- 표 안 숫자, 이미지 속 텍스트가 누락  
- 줄바꿈/공백 오류로 문단 구조가 깨짐  
→ **청킹과 임베딩 전체 품질 저하**

##### **② 단순 청킹(one-size-fits-all) 문제**
문서 구조(제목·섹션·문단)를 고려하지 않은 고정 길이 분할은 다음 문제를 만든다.  
- 문장이 중간에서 끊김  
- 서로 다른 의미가 하나의 청크에 섞임  
- 문단의 논리적 흐름이 사라짐  
→ **검색 정확도와 관련성 저하**

##### **③ 비효율적인 인덱싱 구조**
벡터DB의 인덱스가 적절히 구성되지 않으면  
- 검색 시간이 느려지고  
- 대규모 문서에서 확장성도 떨어집니다.  
→ **응답 지연, 검색 품질 하락**

##### **④ 임베딩 모델의 한계**
임베딩 모델이 도메인 특성을 잘 반영하지 못하면  
- 문서의 의미를 정확히 포착하지 못하고  
- 중요한 문서를 상위에 올리지 못하며  
- 무관한 문서를 반환하는 문제 발생  
→ **검색 Recall 및 Precision 모두 저하**

### 4.2 Retrieval 단계의 한계

##### **① 단일 검색 방식의 한계**
Naive RAG는 대부분 **코사인 유사도 기반 벡터 검색만** 사용한다.  
하지만 단일 검색 방식은 다음과 같은 한계를 가집니다.

- 의미 기반 검색만 사용할 경우  
  → 키워드 중심 문서를 놓칠 수 있음  
- 키워드 중심 검색만 사용할 경우(BM25 등)  
  → 의미적 연관성이 떨어짐  

→ **중요 문서 누락(Recall 감소)** 또는  
→ **원하지 않는 문서 노출(Precision 감소)**

##### **② 쿼리 품질의 한계**
사용자 질문이 모호하거나 문법적으로 비정상일 때  
그대로 임베딩하면 검색 실패로 이어집니다.

예:  
- "그 정책 기준 알려줘" → 맥락 부족  
- "전력수요 올라감 원인?" → 비문  

→ **쿼리 벡터가 문서와 매칭되지 않아 관련 문서를 찾지 못함**

##### **③ 중복 문맥 반환**
청크가 서로 유사한 형태로 과도하게 분할되면 검색 결과에서 중복된 문맥이 여러 개 반환됩니다.

→ LLM이 반복적인 답변을 생성하거나  
→ 중요한 다른 정보가 컨텍스트에서 밀려 사라짐

### 4.3 Generation 단계의 한계

##### **① LLM 기본 성능의 한계 → 할루시네이션**
검색된 문서가 부족하거나,  
검색된 문서와 질문의 의미적 연결이 약하면  
LLM은 훈련 중 학습한 언어적 확률로 추측하기 시작한다.

→ **근거 없는 답변(hallucination) 생성**

##### **② Context Window 한계 → 정보 손실**
LLM은 입력으로 받을 수 있는 토큰 수가 제한되어 있습니다.  
검색된 문서가 많으면 컨텍스트 일부가 잘려나가고  
LLM은 필요한 정보를 충분히 보지 못한다.

→ **불완전한 근거 기반 답변 생성  
→ 문맥 연결 부족**

##### **③ Retrieval된 내용을 활용하지 못하는 모델 특성**
LLM은 기본적으로 문서를 근거로 답을 찾는 QA에 완전 특화된 모델이 아니다.  
특히 도메인 특화 QA 훈련이 없는 모델일수록  
- 컨텍스트를 무시하고  
- 질문만 보고 답을 생성하려는 경향이 강하다.

→ **문서 속 정확한 답이 있어도 활용하지 못하는 문제**

##### **④ 중복 문맥 & 불필요한 정보 → 모델 혼란**
중복된 청크나 장황한 컨텍스트가 많으면  
모델은 어떤 근거를 우선해야 할지 판단하기 어렵다.

→ **반복적/중복적 답변**  
→ **모호한 요약형 답변**  
→ **일관성 부족**

### 4.4 Advanced RAG란 무엇인가?

> **Advanced RAG는 Naive RAG의 단순한 검색 → 생성 구조를 확장하여,  
> 검색 품질을 정교하게 향상시키고, 문맥 전달·정렬·필터링·압축을 최적화하여  
> LLM이 더 정확한 근거 기반 응답을 생성하도록 설계된 RAG 기법들의 집합입니다.**

Advanced RAG는 일반적으로 다음 요소를 포함한다.

- **고급 청킹(Chunking) 기법**  
  (문서 구조 기반, 의미 기반, 재귀적 청킹 등)  
- **Query Transformation**  
  (rewriting, expansion, multi-query)  
- **Hybrid Retrieval**  
  (vector + keyword, 다양한 similarity 조합)  
- **Reranking 및 Retrieval Fusion**  
- **Context Filtering / Compression**  
- **Generator 단계 최적화**  
  (출력 제약, citation 포함 답변 등)

즉, **Advanced RAG는 Naive RAG의 모든 병목을 보완한 실전형 RAG 전략입니다.**

<img src="image/paradigms_of_RAG.png" width="600">